## Text-from-Embedding (Russian SBERT -> Russian BART) - v_hf (с загрузкой на HF)

Восстанавливаем (или перефразируем) текст на русском языке, имея на входе **ровно один 1024-мерный SBERT-вектор**.
Эта версия ноутбука позволяет:
1. Скачивать текстовый файл с Google Drive (URL указан по умолчанию).
2. Автоматически разбивать текст на обучающие примеры (2-5 предложений) с использованием `rusenttokenize`.
3. Обучать модель.
4. Загружать обученную модель и SBERT для инференса один раз для многократного использования.
5. Тестировать генерацию на произвольном введенном тексте с выводом косинусной близости.
6. (Опционально) Загружать обученный проектор модели на Hugging Face Hub.

In [2]:
# @markdown Выполните эту ячейку для установки зависимостей и клонирования репозитория.
# @markdown **Важно**: Если вы хотите использовать свой репозиторий, измените URL ниже.

GIT_REPO_URL = "https://github.com/maxxxsudb/Text-from-Embedding.git" # @param {type:"string"}
# Например: "https://github.com/AI-Guru/text-from-embedding-russian-demo.git"
# Если вы просто тестируете, можете закомментировать клонирование и загрузить файлы вручную.

try:
  import os
  if not os.path.exists('text-from-embedding'):
    print(f'Клонирование репозитория из {GIT_REPO_URL}...')
    !git clone $GIT_REPO_URL text-from-embedding
  else:
    print('Директория text-from-embedding уже существует.')
  %cd text-from-embedding
  print('Установка зависимостей из setup.sh...')
  !bash setup.sh
except Exception as e:
  print(f"Ошибка при клонировании или установке: {e}")
  print("Убедитесь, что GIT_REPO_URL корректен, или вы находитесь в правильной директории, если файлы уже есть.")

print('Установка gdown, rusenttokenize, huggingface_hub...')
!pip install gdown --quiet
!pip install rusenttokenize --quiet
!pip install huggingface_hub --quiet
print('Установленные версии:')
!pip freeze | grep -E "torch|transformers|sentence-transformers|sacrebleu|gdown|rusenttokenize|huggingface-hub"

Клонирование репозитория из https://github.com/maxxxsudb/Text-from-Embedding.git...
Cloning into 'text-from-embedding'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 76 (delta 37), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 63.65 KiB | 3.54 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/text-from-embedding
Установка зависимостей из setup.sh...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [3]:
# @markdown Эта ячейка попытается загрузить необходимые модели и токенизаторы заранее,
# @markdown чтобы выявить проблемы до начала основного процесса.
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
import torch

# --- 1. Токенизатор предложений rusenttokenize ---
print("--- Проверка rusenttokenize ---")
RUSENTTOKENIZE_OK = False
try:
    from rusenttokenize import ru_sent_tokenize
    test_sentences = ru_sent_tokenize("Это тестовое предложение. А это второе.")
    if len(test_sentences) == 2:
        print("rusenttokenize успешно импортирован и работает.")
        RUSENTTOKENIZE_OK = True
    else:
        print("rusenttokenize импортирован, но тестовая токенизация дала неожиданный результат.")
except ImportError:
    print("Ошибка: не удалось импортировать rusenttokenize. Убедитесь, что он установлен в Ячейке 1.")
except Exception as e_rusent:
    print(f"Ошибка при проверке rusenttokenize: {e_rusent}")

if not RUSENTTOKENIZE_OK:
    print("ПРЕДУПРЕЖДЕНИЕ: rusenttokenize не работает корректно. Токенизация предложений в Ячейке 2 будет невозможна этим методом.")
print("-" * 30 + "\n")


# --- 2. SBERT модель (используется в подготовке и инференсе) ---
SBERT_MODEL_TO_CHECK = "sberbank-ai/sbert_large_nlu_ru" # @param {type:"string"}
print(f"--- Проверка SBERT модели: {SBERT_MODEL_TO_CHECK} ---")
sbert_test_model_instance = None
try:
    sbert_test_model_instance = SentenceTransformer(SBERT_MODEL_TO_CHECK)
    print(f"SBERT модель '{SBERT_MODEL_TO_CHECK}' успешно загружена.")
    try:
        dummy_emb = sbert_test_model_instance.encode(["тест"])
        print(f"Размерность эмбеддинга SBERT: {dummy_emb.shape[1]}")
    except Exception as e_emb:
        print(f"Не удалось получить тестовый эмбеддинг от SBERT: {e_emb}")
except Exception as e:
    print(f"Ошибка при загрузке SBERT модели '{SBERT_MODEL_TO_CHECK}': {e}")
finally:
    if sbert_test_model_instance is not None:
        del sbert_test_model_instance
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU кэш очищен после проверки SBERT.")
print("-" * 30 + "\n")

# --- 3. BART модель и токенизатор (используется в обучении и инференсе) ---
BART_MODEL_TO_CHECK = "Den4ikAI/bart_ru_summarization" # @param {type:"string"}
print(f"--- Проверка BART модели и токенизатора: {BART_MODEL_TO_CHECK} ---")
bart_tokenizer_test_instance = None
bart_model_test_instance = None
try:
    bart_tokenizer_test_instance = AutoTokenizer.from_pretrained(BART_MODEL_TO_CHECK)
    print(f"Токенизатор для BART '{BART_MODEL_TO_CHECK}' успешно загружен.")

    bart_model_test_instance = AutoModelForSeq2SeqLM.from_pretrained(BART_MODEL_TO_CHECK)
    print(f"BART модель '{BART_MODEL_TO_CHECK}' успешно загружена.")
    print(f"  d_model BART: {bart_model_test_instance.config.d_model}")
    print(f"  decoder_start_token_id BART: {bart_model_test_instance.config.decoder_start_token_id}")
except Exception as e:
    print(f"Ошибка при загрузке BART: {e}")
finally:
    if bart_tokenizer_test_instance is not None:
        del bart_tokenizer_test_instance
    if bart_model_test_instance is not None:
        del bart_model_test_instance
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU кэш очищен после проверки BART.")
print("-" * 30 + "\n")

print("Предварительная проверка завершена.")
if torch.cuda.is_available():
    print("Финальная очистка GPU кэша в конце ячейки 1.1.")
    torch.cuda.empty_cache()

--- Проверка rusenttokenize ---
rusenttokenize успешно импортирован и работает.
------------------------------

--- Проверка SBERT модели: sberbank-ai/sbert_large_nlu_ru ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

...

Токенизатор для BART 'Den4ikAI/bart_ru_summarization' успешно загружен.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

BART модель 'Den4ikAI/bart_ru_summarization' успешно загружена.
  d_model BART: 1024
  decoder_start_token_id BART: 250021
GPU кэш очищен после проверки BART.
------------------------------

Предварительная проверка завершена.
Финальная очистка GPU кэша в конце ячейки 1.1.


In [4]:
import json
from sentence_transformers import SentenceTransformer
import os
import gdown
import random
from rusenttokenize import ru_sent_tokenize
import torch

# @markdown ---
# @markdown **Настройки скачивания и обработки текста:**
GOOGLE_DRIVE_FILE_URL = "https://drive.google.com/file/d/1sBt-sZZ-7p0WKOb3Usg2JkjCiXZleGXw/view?usp=drive_link" # @param {type:"string"}
LOCAL_TEXT_FILE_NAME = "corpus.txt" # @param {type:"string"}
MIN_SENTENCES_PER_CHUNK = 2 # @param {type:"integer"}
MAX_SENTENCES_PER_CHUNK = 5 # @param {type:"integer"}
TRAIN_SPLIT_RATIO = 0.9 # @param {type:"slider", min:0.1, max:0.95, step:0.05}

# @markdown ---
# @markdown **Параметры SBERT модели (должны совпадать с Ячейкой 1.1 и Ячейкой 4):**
SBERT_MODEL_NAME_PREP = "sberbank-ai/sbert_large_nlu_ru" # @param {type:"string"}
SBERT_DIM_PREP = 1024 # @param {type:"integer"}

OUTPUT_TRAIN_JSONL = "data/train_corpus.jsonl"
OUTPUT_VAL_JSONL = "data/val_corpus.jsonl"

def download_file_from_gdrive(url, output_path):
    try:
        gdown.download(url, output_path, quiet=False, fuzzy=True)
        print(f"Файл успешно скачан и сохранен как {output_path}")
        return True
    except Exception as e:
        print(f"Ошибка при скачивании файла: {e}")
        return False

def split_text_into_sentences_custom(text):
    print("Использование rusenttokenize для разделения на предложения...")
    try:
        sentences = ru_sent_tokenize(text)
        return [s.strip() for s in sentences if s.strip()]
    except Exception as e:
        print(f"Ошибка при использовании ru_sent_tokenize: {e}")
        print("Попытка использовать очень простой fallback метод разделения по точкам и новым строкам.")
        sentences_fallback = [s.strip() for s_para in text.split('\n') for s in s_para.split('.') if s.strip()]
        if sentences_fallback:
            print("ВНИМАНИЕ: Использован грубый метод разделения предложений. Качество может пострадать.")
            return sentences_fallback
        else:
            print("Не удалось разделить текст на предложения даже грубым методом.")
            return []

def create_text_chunks(sentences, min_sent, max_sent):
    chunks = []
    i = 0
    while i < len(sentences):
        num_sentences_in_chunk = random.randint(min_sent, max_sent)
        chunk = sentences[i : i + num_sentences_in_chunk]
        if chunk:
            chunks.append(" ".join(chunk))
        i += num_sentences_in_chunk
    return chunks

def create_jsonl_data(text_chunks, output_path, sbert_model_instance, sbert_dim_check, sbert_model_name_log):
    print(f"Generating embeddings for {output_path} using SBERT model {sbert_model_name_log}...")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    count = 0
    with open(output_path, "w", encoding="utf-8") as f_out:
        batch_size = 32
        for i in range(0, len(text_chunks), batch_size):
            batch_texts = text_chunks[i:i+batch_size]
            embeddings = sbert_model_instance.encode(batch_texts, convert_to_tensor=False, normalize_embeddings=False, show_progress_bar=True)
            for text, vec in zip(batch_texts, embeddings):
                if vec.shape[0] != sbert_dim_check:
                    raise ValueError(f"SBERT model {sbert_model_name_log} produced embeddings of dim {vec.shape[0]}, expected {sbert_dim_check}")
                record = {"embedding": vec.tolist(), "text": text}
                f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
                count += 1
    print(f"Saved {count} records to {output_path}")

# --- Основной пайплайн подготовки данных ---
sbert_for_prep_data_instance = None
try:
    if not GOOGLE_DRIVE_FILE_URL:
        print("URL файла Google Drive не указан. Пропустите эту ячейку или укажите URL.")
    else:
        if download_file_from_gdrive(GOOGLE_DRIVE_FILE_URL, LOCAL_TEXT_FILE_NAME):
            with open(LOCAL_TEXT_FILE_NAME, "r", encoding="utf-8") as f:
                corpus_text = f.read()

            print(f"Объем текста: {len(corpus_text)} символов.")
            all_sentences = split_text_into_sentences_custom(corpus_text)
            print(f"Найдено предложений: {len(all_sentences)}")

            if not all_sentences:
                print("Не удалось извлечь предложения из текста. Проверьте содержимое файла.")
            else:
                text_chunks_for_sbert = create_text_chunks(all_sentences, MIN_SENTENCES_PER_CHUNK, MAX_SENTENCES_PER_CHUNK)
                print(f"Создано текстовых чанков (примеров): {len(text_chunks_for_sbert)}")
                random.shuffle(text_chunks_for_sbert)

                split_idx = int(len(text_chunks_for_sbert) * TRAIN_SPLIT_RATIO)
                train_chunks = text_chunks_for_sbert[:split_idx]
                val_chunks = text_chunks_for_sbert[split_idx:]
                print(f"Обучающих примеров: {len(train_chunks)}, Валидационных примеров: {len(val_chunks)}")

                if not train_chunks or not val_chunks:
                    print("Недостаточно данных для создания обучающей и валидационной выборок. Уменьшите TRAIN_SPLIT_RATIO или увеличьте объем текста.")
                else:
                    print(f"Loading SBERT model for data preparation: {SBERT_MODEL_NAME_PREP}...")
                    sbert_for_prep_data_instance = SentenceTransformer(SBERT_MODEL_NAME_PREP)
                    print("SBERT model loaded.")

                    create_jsonl_data(train_chunks, OUTPUT_TRAIN_JSONL, sbert_for_prep_data_instance, SBERT_DIM_PREP, SBERT_MODEL_NAME_PREP)
                    create_jsonl_data(val_chunks, OUTPUT_VAL_JSONL, sbert_for_prep_data_instance, SBERT_DIM_PREP, SBERT_MODEL_NAME_PREP)

                    if os.path.exists(OUTPUT_TRAIN_JSONL):
                        print("\nСодержимое data/train_corpus.jsonl (первая строка):")
                        !head -n 1 data/train_corpus.jsonl
                    if os.path.exists(OUTPUT_VAL_JSONL):
                        print("\nСодержимое data/val_corpus.jsonl (первая строка):")
                        !head -n 1 data/val_corpus.jsonl
        else:
            print("Не удалось скачать файл. Обучение на основе этого файла невозможно.")
finally:
    if sbert_for_prep_data_instance is not None:
        del sbert_for_prep_data_instance
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU кэш очищен после подготовки данных.")

Downloading...
From: https://drive.google.com/uc?id=1sBt-sZZ-7p0WKOb3Usg2JkjCiXZleGXw
To: /content/text-from-embedding/corpus.txt

100%|██████████| 164k/164k [00:00<00:00, 55.0MB/s]


Файл успешно скачан и сохранен как corpus.txt
Объем текста: 91704 символов.
Использование rusenttokenize для разделения на предложения...


Найдено предложений: 1720
Создано текстовых чанков (примеров): 492
Обучающих примеров: 442, Валидационных примеров: 50
Loading SBERT model for data preparation: sberbank-ai/sbert_large_nlu_ru...
SBERT model loaded.
Generating embeddings for data/train_corpus.jsonl using SBERT model sberbank-ai/sbert_large_nlu_ru...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Saved 442 records to data/train_corpus.jsonl
Generating embeddings for data/val_corpus.jsonl using SBERT model sberbank-ai/sbert_large_nlu_ru...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Saved 50 records to data/val_corpus.jsonl

Содержимое data/train_corpus.jsonl (первая строка):
{"embedding": [0.013756760396063328, 0.03613490238785744, -0.031771205365657806, -0.027862725779414177, 0.001515597803518176, 0.03657111898064613, -0.022991225123405457, 0.010364233516156673, -0.0024229648988693953, 0.00024041994765866548, -0.019849255681037903, 0.03439187631011009, -0.00733420392498374, 0.009413373656570911, -0.01843714900314808, -0.004018677398562431, 0.017695460468530655, 0.02202894724905491, -0.0001821757759898901, 0.10101494938135147, 0.07354699820280075, -0.03470047563314438, -0.02863406576216221, -0.0072351014241576195, 0.0004751217784360051, -0.0009509955416433513, 0.000889926275704056, 0.010615303181111813, -0.05579220503568649, 0.011042419821023941, -0.030348019674420357, -0.050134871155023575, 0.00032898050267249346, -0.015644123777747154, 0.06065744906663895, -0.0019986191764473915, -0.014003844931721687, 0.031418509781360626, -0.0015844270819798112, -0.0021603736

In [5]:
import os
# @markdown Запускаем обучение. Убедитесь, что данные (`data/train_corpus.jsonl` и `data/val_corpus.jsonl`) были успешно созданы в предыдущей ячейке.

# @markdown ---
# @markdown **Параметры обучения (BART и SBERT должны совпадать с Ячейкой 1.1 и 2):**
BART_MODEL_TRAIN = "Den4ikAI/bart_ru_summarization" # @param {type:"string"}
SBERT_DIM_TRAIN_PARAM = 1024 # @param {type:"integer"}
SAVE_DIR_TRAIN = "checkpoints/corpus_run_russian_final" # @param {type:"string"}
EPOCHS_TRAIN = 15 # @param {type:"integer"}
BATCH_SIZE_TRAIN = 4 # @param {type:"integer"}
LEARNING_RATE_TRAIN = 3e-4 # @param {type:"number"}
PROJECTOR_K_TRAIN = 1 # @param {type:"integer"}
PROJECTOR_BOTTLENECK_DIM_TRAIN = 1024 # @param {type:"integer"}
MAX_LEN_TOKENIZER_TRAIN = 128 # @param {type:"integer"}
WARMUP_STEPS_TRAIN = 200 # @param {type:"integer"}

OUTPUT_TRAIN_JSONL_CHECK = "data/train_corpus.jsonl"
OUTPUT_VAL_JSONL_CHECK = "data/val_corpus.jsonl"

if not (os.path.exists(OUTPUT_TRAIN_JSONL_CHECK) and os.path.exists(OUTPUT_VAL_JSONL_CHECK)):
    print(f"Файлы {OUTPUT_TRAIN_JSONL_CHECK} и/или {OUTPUT_VAL_JSONL_CHECK} не найдены. Выполните ячейку №2 для подготовки данных.")
else:
    !python -m src.train \
      --train_jsonl data/train_corpus.jsonl \
      --val_jsonl   data/val_corpus.jsonl \
      --save_dir    $SAVE_DIR_TRAIN \
      --bart_model  $BART_MODEL_TRAIN \
      --sbert_dim   $SBERT_DIM_TRAIN_PARAM \
      --epochs      $EPOCHS_TRAIN \
      --bs          $BATCH_SIZE_TRAIN \
      --lr          $LEARNING_RATE_TRAIN \
      --warmup_steps $WARMUP_STEPS_TRAIN \
      --k           $PROJECTOR_K_TRAIN \
      --proj_bottleneck_dim $PROJECTOR_BOTTLENECK_DIM_TRAIN \
      --max_len     $MAX_LEN_TOKENIZER_TRAIN \
      --max_new_tokens_val 72 \
      --num_beams_val 3 \
      --num_workers 2 \
      --label_smoothing 0.1 \
      --calc_cosine_sim  # <--- ДОБАВЛЕННЫЙ ФЛАГ
  # --sbert_model_name "ваша_sbert_модель" # <--- Укажите, если отличается от дефолта

2025-05-07 16:09:22.998901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746634163.019658    1398 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746634163.025890    1398 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/content/text-from-embedding/src/train.py:28: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(device.type == "cuda"))
Device: cuda
Using BART model: Den4ikAI/bart_ru_summarization
Using SBERT dim: 1024
Loading SBERT model for cosine similarity calculation: sberbank-ai/sbert_large_nlu_ru
SBERT model for evaluation loaded.
Proj

In [6]:
# @markdown Эта ячейка загружает лучший обученный чекпоинт и SBERT-модель один раз.
# @markdown Убедитесь, что обучение в Ячейке 3 завершено и CHECKPOINT_DIR_LOAD указывает на правильную директорию.

import torch
import os
import json
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from src.model import Sbert2Text

loaded_sbert2text_model = None
loaded_sbert_inference_model_instance = None
loaded_bart_tokenizer_for_inference = None
loaded_train_args_for_inference = None

# @markdown ---
# @markdown **Настройки для загрузки моделей (должны соответствовать параметрам обучения):**
CHECKPOINT_DIR_LOAD = "checkpoints/corpus_run_russian_final" # @param {type:"string"}
SBERT_MODEL_NAME_LOAD = "sberbank-ai/sbert_large_nlu_ru" # @param {type:"string"}

device_load = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Устройство для загрузки моделей: {device_load}")

try:
    train_args_path_load = os.path.join(CHECKPOINT_DIR_LOAD, "train_args.json")
    if os.path.exists(train_args_path_load):
        print(f"Загрузка аргументов обучения из {train_args_path_load}")
        with open(train_args_path_load, "r") as f:
            loaded_train_args_for_inference = json.load(f)
    else:
        print(f"ПРЕДУПРЕЖДЕНИЕ: Файл {train_args_path_load} не найден.")
        raise FileNotFoundError(f"{train_args_path_load} not found.")

    CKPT_PATH_LOAD = os.path.join(CHECKPOINT_DIR_LOAD, "best_bleu_model.pt")
    if not os.path.exists(CKPT_PATH_LOAD) and os.path.exists(os.path.join(CHECKPOINT_DIR_LOAD, "last_checkpoint.pt")):
        print(f"Файл {CKPT_PATH_LOAD} не найден. Попытка использовать last_checkpoint.pt...")
        checkpoint_last = torch.load(os.path.join(CHECKPOINT_DIR_LOAD, "last_checkpoint.pt"), map_location="cpu")
        if 'model_state_dict' in checkpoint_last:
            torch.save(checkpoint_last['model_state_dict'], CKPT_PATH_LOAD)
            print(f"model_state_dict из last_checkpoint.pt сохранен как {CKPT_PATH_LOAD}")
        else:
            raise FileNotFoundError(f"'model_state_dict' not in last_checkpoint.pt and {CKPT_PATH_LOAD} not found.")
    elif not os.path.exists(CKPT_PATH_LOAD):
        raise FileNotFoundError(f"Чекпоинт {CKPT_PATH_LOAD} не найден.")

    print(f"Загрузка модели Sbert2Text из {CKPT_PATH_LOAD}...")
    bart_name_load = loaded_train_args_for_inference.get("bart_model", "Den4ikAI/bart_ru_summarization")
    sbert_dim_load = loaded_train_args_for_inference.get("sbert_dim", 1024)
    projector_k_load = loaded_train_args_for_inference.get("k", 1)
    proj_bottleneck_dim_load_val = loaded_train_args_for_inference.get("proj_bottleneck_dim", 1024)
    actual_proj_bottleneck_dim_load = proj_bottleneck_dim_load_val if proj_bottleneck_dim_load_val is not None and proj_bottleneck_dim_load_val > 0 else None

    loaded_sbert2text_model = Sbert2Text(
        bart_name=bart_name_load,
        sbert_dim=sbert_dim_load,
        projector_k=projector_k_load,
        projector_bottleneck_dim=actual_proj_bottleneck_dim_load,
        label_smoothing_factor=0.0
    ).to(device_load)
    loaded_sbert2text_model.load_state_dict(torch.load(CKPT_PATH_LOAD, map_location=device_load))
    loaded_sbert2text_model.eval()
    print("Модель Sbert2Text успешно загружена и переведена в режим eval.")

    loaded_bart_tokenizer_for_inference = AutoTokenizer.from_pretrained(bart_name_load)
    print(f"Токенизатор BART ({bart_name_load}) для инференса загружен.")

    print(f"\nЗагрузка SBERT модели для инференса: {SBERT_MODEL_NAME_LOAD}...")
    loaded_sbert_inference_model_instance = SentenceTransformer(SBERT_MODEL_NAME_LOAD, device=device_load)
    print("SBERT модель для инференса успешно загружена.")

    if loaded_sbert2text_model and loaded_sbert_inference_model_instance and loaded_bart_tokenizer_for_inference:
        print("\nВсе необходимые модели для инференса загружены и готовы к использованию в Ячейке 4.")
    else:
        print("\nПРЕДУПРЕЖДЕНИЕ: Одна или несколько моделей для инференса не были загружены.")

except Exception as e_main_load:
    print(f"ОБЩАЯ ОШИБКА при загрузке моделей для инференса: {e_main_load}")
    loaded_sbert2text_model = None
    loaded_sbert_inference_model_instance = None
    loaded_bart_tokenizer_for_inference = None
    loaded_train_args_for_inference = None
finally:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU кэш очищен после загрузки моделей для инференса.")

Устройство для загрузки моделей: cuda
Загрузка аргументов обучения из checkpoints/corpus_run_russian_final/train_args.json
Загрузка модели Sbert2Text из checkpoints/corpus_run_russian_final/best_bleu_model.pt...
KeyValueProjector (2-Layer MLP): in_dim=1024 -> bottleneck=1024 -> k*d_model=1024
KeyValueProjector trainable parameters: 2.10 M
Sbert2Text: BART model 'Den4ikAI/bart_ru_summarization' (d_model=1024) loaded and frozen.
Projector k=1, SBERT dim=1024
Using decoder_start_token_id for generate: 250021
Модель Sbert2Text успешно загружена и переведена в режим eval.
Токенизатор BART (Den4ikAI/bart_ru_summarization) для инференса загружен.

Загрузка SBERT модели для инференса: sberbank-ai/sbert_large_nlu_ru...
SBERT модель для инференса успешно загружена.

Все необходимые модели для инференса загружены и готовы к использованию в Ячейке 4.
GPU кэш очищен после загрузки моделей для инференса.


In [10]:
import numpy as np
import torch
from torch.nn.functional import cosine_similarity

# @markdown ---
# @markdown **Введите ваш текст для генерации:**
USER_TEXT_INPUT_INFER = "Женщина изменила?" # @param {type:"string"}

# @markdown ---
# @markdown **Параметры генерации (для модели Sbert2Text):**
NUM_BEAMS_INFER = 4 # @param {type:"integer"}
MAX_NEW_TOKENS_INFER = 100 # @param {type:"integer"}
MIN_NEW_TOKENS_INFER = 10 # @param {type:"integer"}
REPETITION_PENALTY_INFER = 1.2 # @param {type:"number"}
NO_REPEAT_NGRAM_SIZE_INFER = 3 # @param {type:"integer"}

device_infer = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def perform_inference_loaded(text_input, sbert2text_model_loaded_local, sbert_model_loaded_local, bart_tokenizer_loaded_local):
    if not text_input.strip():
        print("Пожалуйста, введите текст.")
        return

    # Проверяем, переданы ли модели в функцию (из глобальных переменных)
    if sbert2text_model_loaded_local is None or sbert_model_loaded_local is None or bart_tokenizer_loaded_local is None:
        print("Одна или несколько моделей не загружены (переданы как None). Пожалуйста, выполните Ячейку 3.1.")
        return

    print(f"\n--- Инференс для текста: '{text_input[:100]}...' ---")

    sbert_model_loaded_local.to(device_infer)
    input_sbert_vec_np = sbert_model_loaded_local.encode(text_input, convert_to_tensor=False, normalize_embeddings=True)
    input_sbert_vec_tensor_for_gen = torch.tensor(input_sbert_vec_np, dtype=torch.float).unsqueeze(0).to(device_infer)
    print(f"Форма входного SBERT-вектора для генерации: {input_sbert_vec_tensor_for_gen.shape}")

    print("Генерация текста...")
    sbert2text_model_loaded_local.to(device_infer)
    sbert2text_model_loaded_local.eval()
    with torch.no_grad():
        gen_params = {
            "num_beams": NUM_BEAMS_INFER,
            "max_new_tokens": MAX_NEW_TOKENS_INFER,
            "min_length": MIN_NEW_TOKENS_INFER,
            "repetition_penalty": REPETITION_PENALTY_INFER,
            "no_repeat_ngram_size": NO_REPEAT_NGRAM_SIZE_INFER,
            "early_stopping": True if NUM_BEAMS_INFER > 1 else False,
        }
        out_ids = sbert2text_model_loaded_local.generate(input_sbert_vec_tensor_for_gen, **gen_params)

    generated_texts_list = bart_tokenizer_loaded_local.batch_decode(out_ids, skip_special_tokens=True)
    generated_text = generated_texts_list[0] if generated_texts_list else "Не удалось сгенерировать текст."

    print("\n--- Сгенерированный текст: ---")
    print(generated_text)

    print("\nКодирование сгенерированного текста с помощью SBERT...")
    generated_sbert_vec_np = sbert_model_loaded_local.encode(generated_text, convert_to_tensor=False, normalize_embeddings=True)
    generated_sbert_vec_tensor_for_sim = torch.tensor(generated_sbert_vec_np, dtype=torch.float).unsqueeze(0).to(device_infer)

    input_sbert_vec_for_sim = torch.tensor(input_sbert_vec_np, dtype=torch.float).to(device_infer)
    if input_sbert_vec_for_sim.ndim == 1:
        input_sbert_vec_for_sim = input_sbert_vec_for_sim.unsqueeze(0)

    cos_sim = cosine_similarity(input_sbert_vec_for_sim, generated_sbert_vec_tensor_for_sim, dim=1)

    print("\n--- Оценка качества: ---")
    print(f"Косинусная близость (SBERT вх. vs SBERT ген.): {cos_sim.item():.4f}")

if 'loaded_sbert2text_model' in globals() and loaded_sbert2text_model is not None:
    perform_inference_loaded(USER_TEXT_INPUT_INFER,
                             loaded_sbert2text_model,
                             loaded_sbert_inference_model_instance,
                             loaded_bart_tokenizer_for_inference)
else:
    print("Модели не были загружены в Ячейке 3.1. Пожалуйста, выполните Ячейку 3.1 перед инференсом.")


--- Инференс для текста: 'Женщина изменила?...' ---
Форма входного SBERT-вектора для генерации: torch.Size([1, 1024])
Генерация текста...

--- Сгенерированный текст: ---
Маленький человек, он думает, что не может быть хорошим человеком.

Кодирование сгенерированного текста с помощью SBERT...

--- Оценка качества: ---
Косинусная близость (SBERT вх. vs SBERT ген.): 0.3388


In [ ]:
# @markdown Эта ячейка загружает ТОЛЬКО обученный проектор и конфигурацию модели.
# @markdown BART-декодер НЕ загружается, предполагается, что он будет взят с Hugging Face по имени.

# @markdown ---
# @markdown **Требуется вход в Hugging Face CLI:**
# @markdown Если вы еще не вошли, выполните в новой ячейке кода: `!huggingface-cli login`
# @markdown или `from huggingface_hub import notebook_login; notebook_login()`

# @markdown ---
# @markdown **Настройки для загрузки:**
HF_MODEL_ID = "YourHuggingFaceUsername/YourSbert2TextProjectorModelName" # @param {type:"string"}
COMMIT_MESSAGE_HF = "Upload trained SBERT2Text projector and config" # @param {type:"string"}
CHECKPOINT_DIR_HF_UPLOAD = "checkpoints/corpus_run_russian_final" # @param {type:"string"}
CREATE_PRIVATE_REPO_HF = False # @param {type:"boolean"}

import os
import json
import torch
from huggingface_hub import HfApi, CommitOperationAdd, create_repo
# Импортируем весь модуль, чтобы избежать проблем с относительным импортом в Colab ячейке
import src.model

def upload_projector_to_hf(hf_model_id, checkpoint_dir, commit_message, private_repo):
    api = HfApi()
    print(f"Начинаем процесс загрузки в репозиторий: {hf_model_id}")

    try:
        repo_url = create_repo(hf_model_id, private=private_repo, exist_ok=True, repo_type="model")
        print(f"Репозиторий {hf_model_id} создан или уже существует: {repo_url}")
    except Exception as e_repo:
        print(f"Ошибка при создании репозитория {hf_model_id}: {e_repo}")
        return

    operations = []
    train_args_hf_local = None
    temp_files_to_clean = []

    # --- 1. train_args.json ---
    train_args_path_hf = os.path.join(checkpoint_dir, "train_args.json")
    if os.path.exists(train_args_path_hf):
        operations.append(CommitOperationAdd(path_in_repo="train_args.json", path_or_fileobj=train_args_path_hf))
        print(f"[OK] Файл train_args.json добавлен для загрузки.")
        try:
            with open(train_args_path_hf, "r") as f_args_hf:
                 train_args_hf_local = json.load(f_args_hf)
        except Exception as e_read_args:
             print(f"Не удалось прочитать train_args.json: {e_read_args}")
    else:
        print(f"ПРЕДУПРЕЖДЕНИЕ: {train_args_path_hf} не найден. Конфигурация модели не будет загружена.")

    # --- 2. projector.pt ---
    best_ckpt_path_hf = os.path.join(checkpoint_dir, "best_bleu_model.pt")
    projector_state_dict_path = "projector_only_state_dict.pt"
    temp_files_to_clean.append(projector_state_dict_path)

    if os.path.exists(best_ckpt_path_hf) and train_args_hf_local:
        print("Извлечение весов проектора из лучшего чекпоинта...")
        try:
            bart_name_for_proj_extraction = train_args_hf_local.get("bart_model")
            sbert_dim_for_proj_extraction = train_args_hf_local.get("sbert_dim")
            k_for_proj_extraction = train_args_hf_local.get("k")
            bottleneck_val_for_proj = train_args_hf_local.get("proj_bottleneck_dim")
            bottleneck_for_proj_extraction = bottleneck_val_for_proj if bottleneck_val_for_proj is not None and bottleneck_val_for_proj > 0 else None

            # Используем имя класса напрямую из импортированного модуля
            temp_full_model = src.model.Sbert2Text(
                bart_name=bart_name_for_proj_extraction,
                sbert_dim=sbert_dim_for_proj_extraction,
                projector_k=k_for_proj_extraction,
                projector_bottleneck_dim=bottleneck_for_proj_extraction,
                label_smoothing_factor=0.0
            )
            temp_full_model.load_state_dict(torch.load(best_ckpt_path_hf, map_location="cpu"))

            projector_weights = temp_full_model.projector.state_dict()
            del temp_full_model
            if torch.cuda.is_available(): torch.cuda.empty_cache()

            if projector_weights:
                torch.save(projector_weights, projector_state_dict_path)
                operations.append(CommitOperationAdd(path_in_repo="projector.pt", path_or_fileobj=projector_state_dict_path))
                print(f"[OK] Веса проектора (projector.pt) добавлены для загрузки.")
            else:
                print("Не удалось извлечь веса проектора.")
        except Exception as e_proj:
            print(f"Ошибка при извлечении или сохранении весов проектора: {e_proj}")
    else:
        if not os.path.exists(best_ckpt_path_hf):
             print(f"ПРЕДУПРЕЖДЕНИЕ: Чекпоинт {best_ckpt_path_hf} не найден.")
        if not train_args_hf_local:
             print(f"ПРЕДУПРЕЖДЕНИЕ: Аргументы обучения не были загружены.")
        print("Веса проектора не будут загружены.")

    # --- 3. README.md ---
    print("Генерация README.md...")
    readme_path = "README_HF_MODEL.md"
    temp_files_to_clean.append(readme_path)
    # Формируем контент README более безопасно
    readme_lines = [
        "---",
        "license: apache-2.0",
        "language: ru",
        "tags:",
        "- text-generation",
        "- sbert-to-text",
        "- projector",
        f"- base_bart_model: {train_args_hf_local.get('bart_model', 'N/A') if train_args_hf_local else 'N/A'}",
        "---",
        "",
        f"# SBERT2Text Projector: {hf_model_id}",
        "",
        "Это репозиторий для **обученного проектора** модели Sbert2Text.",
        "Проектор преобразует SBERT-эмбеддинг в память для BART-декодера с целью генерации текста.",
        "",
        f"**Базовая BART модель (не включена сюда, загружается отдельно):** `{train_args_hf_local.get('bart_model', 'N/A') if train_args_hf_local else 'N/A'}`",
        "",
        "**Конфигурация проектора (из `train_args.json`):**",
        f"- SBERT_dim: {train_args_hf_local.get('sbert_dim', 'N/A') if train_args_hf_local else 'N/A'}",
        f"- Projector k: {train_args_hf_local.get('k', 'N/A') if train_args_hf_local else 'N/A'}",
        f"- Projector bottleneck_dim: {train_args_hf_local.get('proj_bottleneck_dim', 'N/A') if train_args_hf_local else 'N/A'}",
        "",
        "## Как использовать (примерный сценарий):",
        "",
        "```python",
        "import torch",
        "from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig",
        "from huggingface_hub import hf_hub_download",
        "import json",
        "",
        "# Предположим, у вас есть код для Sbert2Text и KeyValueProjector в src.model",
        "from src.model import Sbert2Text",
        "",
        f"hf_repo_id = \"{hf_model_id}\" # Ваш репозиторий",
        "",
        "# 1. Загрузить train_args.json (или config.json, если вы его создадите)",
        "train_args_file = hf_hub_download(repo_id=hf_repo_id, filename=\"train_args.json\")",
        "with open(train_args_file, 'r') as f:",
        "    train_args = json.load(f)",
        "",
        "bart_model_name = train_args.get('bart_model')",
        "sbert_dim = train_args.get('sbert_dim')",
        "projector_k = train_args.get('k')",
        "proj_bottleneck_dim_val = train_args.get('proj_bottleneck_dim')",
        "actual_proj_bottleneck_dim = proj_bottleneck_dim_val if proj_bottleneck_dim_val is not None and proj_bottleneck_dim_val > 0 else None",
        "",
        "# 2. Инициализировать модель Sbert2Text",
        "# Важно: BART модель будет загружена и заморожена внутри Sbert2Text",
        "sbert2text_model = Sbert2Text(",
        "    bart_name=bart_model_name,",
        "    sbert_dim=sbert_dim,",
        "    projector_k=projector_k,",
        "    projector_bottleneck_dim=actual_proj_bottleneck_dim",
        ")",
        "",
        "# 3. Загрузить веса для проектора",
        "projector_weights_file = hf_hub_download(repo_id=hf_repo_id, filename=\"projector.pt\")",
        "projector_state_dict = torch.load(projector_weights_file, map_location='cpu')",
        "sbert2text_model.projector.load_state_dict(projector_state_dict)",
        "sbert2text_model.eval()",
        "",
        f"print(f\"Модель Sbert2Text с проектором из {hf_model_id} готова к использованию.\")",
        "",
        "# 4. Подготовьте ваш SBERT вектор (например, размером [1, sbert_dim])",
        "# sbert_vector = torch.randn(1, sbert_dim)",
        "",
        "# 5. Генерация",
        "# generated_ids = sbert2text_model.generate(sbert_vector, num_beams=4, max_new_tokens=50)",
        "# tokenizer_bart = AutoTokenizer.from_pretrained(bart_model_name)",
        "# text = tokenizer_bart.decode(generated_ids[0], skip_special_tokens=True)",
        "# print(text)",
        "```"
    ]
    try:
        with open(readme_path, "w", encoding="utf-8") as f_readme:
            f_readme.write("\n".join(readme_lines))
        operations.append(CommitOperationAdd(path_in_repo="README.md", path_or_fileobj=readme_path))
        print("[OK] Файл README.md добавлен для загрузки.")
    except Exception as e_readme:
        print(f"Ошибка при создании README.md: {e_readme}")

    # --- 4. Коммит и загрузка ---
    if operations:
        try:
            print(f"Загрузка {len(operations)} файлов в репозиторий {hf_model_id}...")
            commit_info = api.create_commit(
                repo_id=hf_model_id,
                operations=operations,
                commit_message=commit_message,
                repo_type="model"
            )
            print(f"Проектор и конфигурация успешно загружены! Commit: {commit_info.oid}")
            print(f"Ссылка: https://huggingface.co/{hf_model_id}")
        except Exception as e_commit:
            print(f"Ошибка при коммите на Hugging Face Hub: {e_commit}")
    else:
        print("Нет файлов для загрузки.")

    # --- 5. Очистка временных файлов ---
    print("Очистка временных файлов...")
    for temp_file in temp_files_to_clean:
         if os.path.exists(temp_file):
              try:
                  os.remove(temp_file)
              except Exception as e_clean:
                  print(f"Не удалось удалить временный файл {temp_file}: {e_clean}")
    print("Очистка завершена.")

# --- Вызов функции загрузки ---
if not HF_MODEL_ID or "YourHuggingFaceUsername" in HF_MODEL_ID or "YourSbert2TextProjectorModelName" in HF_MODEL_ID:
    print("Пожалуйста, установите корректный HF_MODEL_ID (имя вашего репозитория на Hugging Face).")
    print("Пример: 'myusername/my-sbert-projector'.")
else:
    print("\nДля загрузки на Hugging Face может потребоваться аутентификация.")
    print("Если вы еще не вошли, выполните в отдельной НОВОЙ ячейке:")
    print("from huggingface_hub import notebook_login; notebook_login()")
    print("и следуйте инструкциям для ввода токена.")
    print("ИЛИ используйте `!huggingface-cli login` в терминале.")
    # Перед запуском этой ячейки с раскомментированной функцией, убедитесь, что вы ЗАЛОГИНЕНЫ.
    # upload_projector_to_hf(HF_MODEL_ID, CHECKPOINT_DIR_HF_UPLOAD, COMMIT_MESSAGE_HF, CREATE_PRIVATE_REPO_HF)
    print("\nЧтобы выполнить загрузку, убедитесь, что вы аутентифицированы, раскомментируйте вызов функции upload_projector_to_hf в этой ячейке и запустите ее снова.")